In [21]:
import numpy as np
from tqdm import tqdm
import math
import random
import matplotlib.pyplot as plt
import statistics
from sklearn.preprocessing import normalize
from sklearn.utils.validation import check_is_fitted
from sklearn.base import BaseEstimator, DensityMixin
import time

In [2]:
#parameters
from scipy.io import loadmat
stocks = loadmat('data/data_490_1000.mat')

stocks_shape = stocks['A'].shape
number_of_stocks = stocks_shape[0]
number_of_trades = stocks_shape[1]
num_of_simulations = 1

In [3]:
#calculating the loss
def loss(x, r_t):
    return -np.log(np.dot(x, r_t))

# revanue in time t
def revanue(x, r_t, wealth):
    return np.dot(x, r_t) * wealth

# normalize vector
def normalize_func(x):
    sum_x = np.sum(x)
    x = x/sum_x
    return x

# gradient vector
def gradient(x, r_t):
    return -r_t * (1/np.dot(x, r_t))


#plot
def plot(final_OGD, final_EGD, final_ONS, final_rebalnce, final_fixed_stock, number_of_trades, num_of_simulations):
    if final_OGD is not 0:
        plt.plot(range(1, number_of_trades+1), final_OGD /  num_of_simulations, 'green')
    if final_EGD is not 0:
        plt.plot(range(1, number_of_trades+1), final_EGD /  num_of_simulations, 'red')
    if final_ONS is not 0:
        plt.plot(range(1, number_of_trades+1), final_ONS /  num_of_simulations, 'blue')
    if final_rebalnce is not 0:
        plt.plot(range(1, number_of_trades+1), final_rebalnce /  num_of_simulations, 'grey')
    if final_fixed_stock is not 0:
        plt.plot(range(1, number_of_trades+1), final_fixed_stock /  num_of_simulations, 'black')

    plt.legend(['OGD', 'EGD', 'ONS'
                , 'final_rebalnce', 'final_fixed_stock'], loc='best')
    plt.yscale('log')
    plt.xscale('log')
    plt.grid(True)
    plt.ylabel('Wealth')
    plt.xlabel('number_of_trades')
    plt.title('ORPS methods comparison')
    plt.savefig('ORPS methods comparison.png')
    plt.show()

In [4]:
#get r materix
def get_r_matrix(number_of_stocks, number_of_trades, stocks):
    r_matrix = np.zeros([number_of_stocks, number_of_trades-1])
    for stock in range(number_of_stocks):
        for trade in range(number_of_trades-1):
            r_matrix[stock][trade] = (stocks['A'][stock][trade + 1])/(stocks['A'][stock][trade])
    return r_matrix

#fixed_stock_solution
def best_fixed_stock_solution_func(number_of_trades, number_of_stocks, r_matrix):
    best_fixed_stock_solution = np.zeros(number_of_trades)
    for trade in range(number_of_trades-1):
        max = None
        for stock in range(number_of_stocks):
            if max == None or r_matrix[stock][trade] > max:
                best_fixed_stock_solution[trade] = stock
                max = r_matrix[stock][trade]
    return best_fixed_stock_solution

# best_rebalnce_solution
def best_rebalnce_solution(number_of_trades, r_matrix, number_of_stocks):
    best_fixed_stock_solution = best_fixed_stock_solution_func(number_of_trades, number_of_stocks, r_matrix)
    best_rebalnce_solution_wealth = np.zeros(number_of_trades)
    best_rebalnce_solution_wealth [0] = 1
    for trade in range(1,number_of_trades):
        best_rebalnce_solution_wealth[trade] = best_rebalnce_solution_wealth[trade-1] * r_matrix[int(best_fixed_stock_solution[trade-1])][trade-1]
    return best_rebalnce_solution_wealth

# best fixed stock
def fixed_stock_func(number_of_stocks, number_of_trades, r_matrix):
    all_stocks_wealth = np.zeros([number_of_stocks, number_of_trades])
    for stock in range(number_of_stocks):
        for trade in range(number_of_trades):
            if (trade == 0):
                all_stocks_wealth[stock][trade] = 1
                continue
            all_stocks_wealth[stock][trade] = all_stocks_wealth[stock][trade-1] * r_matrix[stock][trade-1]
    max = None
    index = None
    for stock in range(number_of_stocks):
        if ((max == None) or (all_stocks_wealth[stock][number_of_trades-1]>max)):
            max = all_stocks_wealth[stock][number_of_trades-1]
            index = stock    
    return all_stocks_wealth[index, :]

In [5]:
#Online_gradient_descent
def Online_gradient_descent(x, r_matrix, number_of_trades, step):
    scores_OGD = np.zeros(number_of_trades)
    scores_OGD[0] += 1
    for i in range(1, number_of_trades):
        x = x - step * gradient(x,r_matrix[:,i-1])
        x = normalize_func(x)
        scores_OGD[i] += revanue(x, r_matrix[:,i-1], scores_OGD[i-1])
    return scores_OGD

In [6]:
#denominator of the equation
def exp_stocks_t(x, rt, number_of_stocks, step):
    sum = 0
    for stock in range(number_of_stocks):
        sum += x[stock]* math.exp(-step*rt[stock])
    return sum

#Online Exponentiated Gradient
def Online_Exponentiated_Gradient(x, r_matrix, number_of_trades, number_of_stocks, step):
    scores_OEG = np.zeros(number_of_trades)
    scores_OEG[0] += 1
    for i in range(1, number_of_trades):
        denominator = exp_stocks_t(x, r_matrix[:,i-1], number_of_stocks, step)
        for stock in range(number_of_stocks): 
            x[stock] = (x[stock] * math.exp(-step * r_matrix[stock,i-1])) / denominator
        x = normalize_func(x)
        scores_OEG[i] += revanue(x, r_matrix[:,i-1], scores_OEG[i-1])
    return scores_OEG

In [7]:
#projection on yt+1
def projection(A, eta_min, y, number_of_stocks):
    I = np.eye(number_of_stocks, dtype=np.float32)
    x = I[0]
    for t in range(1, number_of_stocks + 1):
        eta = 2 / (1 + t)
        if eta < eta_min:
            break
        gt = 2 * np.dot(A, x - y)
        imin = np.argmin(gt)
        vt = I[imin]
        x = x + eta * (vt - x)
    return x

#online_newton_step
def online_newton_step(number_of_trades, number_of_stocks, r_matrix, x, D, G):
    scores_ONS = np.zeros(number_of_trades)
    scores_ONS[0] += 1
    gamma = 0.5 *  (1 / (4 * G * D))
    epsilion = 1 / ((gamma ** 2) * (D**2))
    A = epsilion * np.eye(number_of_stocks)
    Ainv = (1 / epsilion) * np.eye(number_of_stocks)
    for trade in range(1, number_of_trades):
        r_t = r_matrix[ : , trade - 1]
        scores_ONS[trade] = revanue(x, r_t, scores_ONS[trade-1])
        grad = gradient(x,r_t)
        grad_matrix = np.outer(grad, grad)
        A = A + grad_matrix
        Ainv = Ainv - (np.dot(Ainv, np.dot(grad_matrix, Ainv)) / (1 + np.dot(grad, np.dot(Ainv, grad))))
        yt = x - (1 / gamma) * np.dot(Ainv, grad)
        eta_min = 0.01
        x = projection(A, eta_min, yt, number_of_stocks)
    return scores_ONS


In [54]:
final_OGD = np.zeros(number_of_trades)
final_EGD = np.zeros(number_of_trades)
final_ONS = np.zeros(number_of_trades)
final_rebalnce = np.zeros(number_of_trades)
final_fixed_stock = np.zeros(number_of_trades)
for i in tqdm(range(num_of_simulations),total=num_of_simulations):
    x = np.zeros(number_of_stocks) + 1 
    x = normalize_func(x)
    r_matrix = get_r_matrix(number_of_stocks, number_of_trades, stocks)
    T = number_of_trades
    D = math.sqrt(2)
    G = np.max(np.linalg.norm(r_matrix, axis=0) / np.sum(r_matrix, axis=0))
    OGD_step = D / (G * math.sqrt(T))
    OEG_step = OGD_step

    #try
#     sim = 1000
#     temp_x = rebalnce_solution(x, number_of_trades, r_matrix, number_of_stocks,sim, step)
#     rebalnce_3 = rebalnce_value(number_of_trades, temp_x, r_matrix)
#     print(rebalnce_3[999])
#     raise
    
#     OGD = Online_gradient_descent(x, r_matrix, number_of_trades, OGD_step)
#     EGD = Online_Exponentiated_Gradient(x, r_matrix, number_of_trades, number_of_stocks, OEG_step)
#     ONS = online_newton_step(number_of_trades, number_of_stocks, r_matrix, x, D, G)
#     rebalnce = best_rebalnce_solution(number_of_trades, r_matrix, number_of_stocks)
    
    matan = rebalnce_solution(x, number_of_trades, r_matrix, number_of_stocks)
    print(matan)
    raise
    
    fixed_stock = fixed_stock_func(number_of_stocks, number_of_trades, r_matrix)
    final_OGD += OGD
    final_EGD += EGD
    final_ONS += ONS
    final_rebalnce += rebalnce
    final_fixed_stock += fixed_stock
    
    final_rebalnce2 = 0
    
plot(final_OGD, final_EGD, final_ONS, final_rebalnce2, final_fixed_stock, number_of_trades, num_of_simulations)

  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: 'staticmethod' object is not callable

# temp reblance, not final

In [5]:
# ### temp reblance, not final

# def total_gradient(x, r_matrix, number_of_trades):
#     new_x = 0
#     for i in range(number_of_trades-1):
#         new_x += gradient(x,r_matrix[:,i])
#     return new_x


# # def rebalnce_solution(x, number_of_trades, r_matrix, number_of_stocks,sim, step):
# #     for i in range(1, sim):
# #         grad = total_gradient(x, r_matrix, number_of_trades)
# # #         loss = total_loss(x, r_matrix)
# #         x = x - (step * grad)
# #         x = normalize_func(x)
# #     return x

# def rebalnce_value(number_of_trades, x, r_matrix):
#     scores_rebalnce = np.zeros(number_of_trades)
#     scores_rebalnce[0] += 1
#     for i in range(1, number_of_trades):
#         scores_rebalnce[i] = scores_rebalnce[i-1] * (np.dot(np.transpose(x), r_matrix[:,i-1]))
#     return scores_rebalnce
    

In [27]:
# final_ONS[998]

41.7981597754372

In [26]:
# final_fixed_stock[998]

32.486486486486605

In [42]:
    R = np.transpose(r_matrix)
    model = BestFixedRebalancingPortfolio(continuous=True, max_iter=25)
    model.fit(R)


In [53]:
# def wealth(r_matrix, p_):
#     R = np.transpose(r_matrix)
#     p = p_
#     return np.cumprod(np.sum(R * p, axis=1), axis=0)
def grad_fn(R, x, number_of_trades):
    a = np.dot(R, x)
    a = - 1 / a
    a = np.reshape(a, (number_of_trades, -1))
    return np.sum(R * a, axis=0)

def _fit(R, number_of_stocks, number_of_trades, max_iter, eta_min):
    # R is the reward (price ratio of day t+1 to day t) per asset
    R = R.astype(np.float32)
    I = np.eye(number_of_stocks, dtype=np.float32)
    # pt is the current iterate
    pt = I[0]
    max_iter = d if max_iter is None else max_iter
    for t in range(1, max_iter + 1):
        eta = 2 / (1 + t)
        if eta < eta_min:
            break
        # rt is the current return vector
        # Gradient of the function we're optimizing
        gt = grad_fn(R, pt, number_of_trades)
        # Solve arg min_{v in V(S)} <v, gt>, where V(S) are the extreme
        # points of the Simplex and <.,.> is an inner product.
        # Solving this is equivalent to taking the standard basis vector
        # which selects the minimal element from gt.
        imin = np.argmin(gt)
        vt = I[imin]
        pt = pt + eta * (vt - pt)
    return pt        

    
def fit(R, continuous, number_of_stocks, number_of_trades, max_iter, eta_min):
    if not continuous:
        p_ = _fit(R, number_of_stocks, number_of_trades, max_iter, eta_min)
    else:
        P_ = np.empty_like(R)
        for t in range(R.shape[0]):
            P_[t] = _fit(R[0:t+1], number_of_stocks, number_of_trades, max_iter, eta_min)
        p_ = P_[-1]
    return p_

def rebalnce_solution(x, number_of_trades, r_matrix, number_of_stocks):
    max_iter = 25
    continuous = True
    eta_min=0.
    R = np.transpose(r_matrix)
    x = fit(R, continuous, number_of_stocks, number_of_trades, max_iter, eta_min)
    return x 

In [37]:
score = np.zeros(number_of_trades)
score[0] = 1
for trade in range (1, number_of_trades):
    score[trade] = score[trade-1] * np.dot(matan, r_matrix[:,trade-1])

In [ ]:
score

In [14]:
class BestFixedRebalancingPortfolio(BaseEstimator, DensityMixin):
    r"""
    Finds the best fixed (in hindsight) rebalancing portfolio
    (distribution over assets). Note that this model is NOT an online
    optimization algorithm since it optimizes over an entire asset dataset
    each step, not sample by sample.

    Solves the optimization problem:
        \arg\min_{x in S} \sum_{t=1}^{T} f_t(x)
    where
        f_t(x) = -\log(r_t^T x)

    Attributes:
        p_: The fitted portfolio
        P_: The fitted portfolio at each time point (only if continuous=True)
    """

    def __init__(self, eta_min=0., max_iter=None, continuous=False):
        """
        :param eta_min: Stop optimization if step size is smaller than this.
        :param max_iter: Maximum number of iterations to fit for. Set to
        None to use the max_iter=d, the dimension of the input.
        :param continuous: Whether to fit continuously to each time point or
        only to the entire dataset (final time).
        """
        super().__init__()
        self.eta_min = eta_min
        self.max_iter = max_iter
        self.continuous = continuous

    @staticmethod
    def grad_fn(R, x):
        T, d = R.shape

        # Grad of -log(r x) is
        #   r * (-1 / (r^T x)) = r * a
        # Where a is a scalar
        a = np.dot(R, x)
        a = - 1 / a
        a = np.reshape(a, (T, -1))

        return np.sum(R * a, axis=0)

    def fit(self, R: np.ndarray):
        """
        Fits a rebalancing constant portfolio, which is a point within the
        unit simplex to the stock price data in R.
        :param R: Array of shape (T,d) where T is the number of time points
        (e.g. days) and d is the number of different assets. Each entry is
        the asset price-ratio between the current and previous time.
        :return: self
        """
        if not self.continuous:
            self.p_ = self._fit(R)
        else:
            self.P_ = np.empty_like(R)
            for t in range(R.shape[0]):
                self.P_[t] = self._fit(R[0:t+1])
            self.p_ = self.P_[-1]

    def _fit(self, R: np.ndarray):
        # R is the reward (price ratio of day t+1 to day t) per asset
        assert R.ndim == 2
        T, d = R.shape
        R = R.astype(np.float32)

        I = np.eye(d, dtype=np.float32)

        # pt is the current iterate
        pt = I[0]
        max_iter = d if self.max_iter is None else self.max_iter
        for t in range(1, max_iter + 1):
            eta = 2 / (1 + t)
            if eta < self.eta_min:
                break

            # rt is the current return vector
            # Gradient of the function we're optimizing
            gt = self.grad_fn(R, pt)

            # Solve arg min_{v in V(S)} <v, gt>, where V(S) are the extreme
            # points of the Simplex and <.,.> is an inner product.
            # Solving this is equivalent to taking the standard basis vector
            # which selects the minimal element from gt.
            imin = np.argmin(gt)
            vt = I[imin]

            pt = pt + eta * (vt - pt)

        return pt

    def wealth(self, R: np.ndarray):
        """
        Calculate the wealth of the algorithm at each trading round.
        :param R: Array of shape (T,d) where T is the number of time points
        (e.g. days) and d is the number of different assets. Each entry is
        the asset price-ratio between the current and previous time.
        :return: Array of shape (T,), containing wealth of the algorithm at
        each trading round.
        """
        assert R.ndim == 2
        check_is_fitted(self, ['p_'])
        p = self.p_
        assert p.shape[0] == R.shape[1]

        return np.cumprod(np.sum(R * p, axis=1), axis=0)